In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
cd /content/drive/MyDrive/'Colab Notebooks'/'yai_9기 겨울방학'/ToyProject

In [ ]:
!pip upgrade ignite

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

In [ ]:
diamond_df = pd.read_csv('./data/images/total.csv')
diamond_df_drop = diamond_df.drop_duplicates(['Id'], keep = 'last')
names = os.listdir('./data/images')
names = list(map(lambda s: s.rstrip('.jpg'),names))
print("csv 중복제거전 중복제거후 : ",len(diamond_df), len(diamond_df_drop))
print("이미지 개수",len(os.listdir('./data/images')))
print("이미지 파일명 \n",names)

idxes = []
for i in range(len(diamond_df_drop)):
    if diamond_df_drop['Id'].iloc[i] not in names:
        idxes.append(diamond_df_drop.index[i])
print("제거돼야할 index들 : ", idxes)
diamond_df_drop.drop(idxes)
# idxes = diamond_df_drop[diamond_df_drop['Id'] not in names].index
# diamond_df_drop = diamond_df_drop(idxes)
# print("이미지 파일 없는 것 제거 후",len(diamond_df_drop))


In [ ]:
names = os.listdir('./data/images')
sizes = []
imageSize = []
for name in names:
    if name == 'total.csv':
        continue
    image = Image.open('./data/images/' + name)
    image = np.array(image)
    sizes.append(image.shape)
    if image.shape == (508, 646, 3):
        imageSize.append(name)

sizes = set(sizes)
print(sizes)


In [ ]:
class DiamondDataset(Dataset):
    def __init__(self, path, transform, train = True):
        self.transform = transform
        self.path = path
        diamond_df = pd.read_csv(path + '/total.csv')
        diamond_df = diamond_df.drop_duplicates(['Id'], keep = 'last')

        names = os.listdir(path)
        names = list(map(lambda s: s.rstrip('.jpg'),names))

        idxes = []
        for i in range(len(diamond_df)):
            if diamond_df['Id'].iloc[i] not in names:
                idxes.append(diamond_df.index[i])
        
        diamond_df = diamond_df.drop(idxes)
        self.diamond_df = diamond_df
        
        
    def __len__(self):
        return len(self.diamond_df)

    def __getitem__(self, idx):
        price = self.diamond_df['Price'].iloc[idx]
        name =  self.diamond_df['Id'].iloc[idx]
        image = Image.open(self.path +'/' + name + '.jpg')

        if self.transform:
            image = self.transform(image)
        return torch.tensor(image, dtype = torch.float), torch.tensor(price, dtype = torch.float)

In [ ]:
!pwd

In [ ]:
dataset = DiamondDataset(path = './data/images',
                         transform = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5,0.5,0.5),
                                                          (0.5,0.5,0.5))                          
                                      ]))
image, price = next(iter(dataset))

In [ ]:
plt.imshow(transforms.ToPILImage()(image*0.5+0.5)) # Normalize 처리때문에 복구
print(price)

In [ ]:
dataloader= DataLoader(dataset=dataset,
                       batch_size=16,
                       shuffle=True,
                       num_workers=4,
                       )
images, labels = next(iter(dataloader))
print(images.shape)
plt.figure(figsize=(12,12))

for n, (image, label) in enumerate(zip(images, labels), start=1):
    plt.subplot(4,4,n)
    plt.imshow(transforms.ToPILImage()(image*0.5+0.5))  # Normalize 처리때문에 복구
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# 모델
resnet18 = torchvision.models.resnet18(pretrained=False)
resnet18.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet18.fc = nn.Sequential(
    nn.Linear(in_features = 512, out_features = 120, bias = True),
    nn.ReLU(),
    nn.Linear(in_features = 120, out_features = 1, bias = True),
    nn.ReLU()    
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

epoch = 30
LEARNING_RATE = 0.0001
batch_size = 64

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(resnet18.parameters(), lr=LEARNING_RATE)

In [ ]:
dataset = DiamondDataset(path = './data/images',
                transform = transforms.Compose([
                            transforms.Resize((300,300)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,0.5,0.5),
                                                 (0.5,0.5,0.5))                          
                            ]))
length = len(dataset)
train_set, test_set = random_split(dataset, [int(length*0.8),length - int(length*0.8)])
train_loader= DataLoader(dataset=train_set,
                       batch_size=batch_size,
                       shuffle=True,
                       num_workers=1,
                       )
test_loader = DataLoader(dataset = test_set,
                         batch_size = batch_size,
                         shuffle = True,
                         num_workers = 1)

In [ ]:
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [ ]:
for epoch in range(epoch):
    running_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        pred = resnet18(images)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss
    avg_loss = running_loss/len(train_loader)
    print(f'[Epoch : {epoch}] average loss : {avg_loss}')

In [ ]:
resnet18.eval()
criterion = nn.MSELoss().to(device)
ss_tot = 0
ss_res = 0
with torch.no_grad():
    for images, prices in test_loader:
        images =images.to(device)
        prices = prices.to(device)
        prediction = resnet18(images).to(device)
        prices_mean = torch.mean(prices)
        ss_tot += torch.sum((prices - prices_mean) ** 2)
        ss_res = torch.sum((prices - prediction) ** 2)
    
    accuracy = 1 - ss_tot/ss_res        
    print(f"Accuracy: {accuracy*100:.2f}%")